In [ ]:
# Gerekli kütüphaneleri yükleyin
!pip install -q librosa scikit-learn pandas xgboost

In [ ]:
# Gerekli kütüphaneleri yükleyin
!pip install -q librosa scikit-learn pandas xgboost

# Gerekli kütüphaneleri içe aktarın
import os
import numpy as np
import pandas as pd
import librosa
import librosa.effects
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
import joblib
from google.colab import files

# Veri setinin ana yolunu ayarlayın.
# Bu kodun çalışması için bu klasörlerin Google Colab'a yüklenmiş olması gerekir.
data_path = '/content/data/'

# --- Fonksiyonlar ---

def extract_features(file_path):
    """Bir ses dosyasından öznitelik (feature) çıkarır."""
    try:
        y, sr = librosa.load(file_path, duration=3, sr=None)

        # Sadece sessizlik kısımlarını budamak için
        y_trimmed, _ = librosa.effects.trim(y)

        # Öznitelik çıkarma
        mfccs = np.mean(librosa.feature.mfcc(y=y_trimmed, sr=sr, n_mfcc=20).T, axis=0)
        chroma = np.mean(librosa.feature.chroma_stft(y=y_trimmed, sr=sr).T, axis=0)
        mel = np.mean(librosa.feature.melspectrogram(y=y_trimmed, sr=sr).T, axis=0)
        contrast = np.mean(librosa.feature.spectral_contrast(y=y_trimmed, sr=sr).T, axis=0)
        tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(y_trimmed), sr=sr).T, axis=0)

        # Tüm öznitelikleri birleştir
        features = np.hstack([mfccs, chroma, mel, contrast, tonnetz])
        return features
    except Exception as e:
        print(f"Hata: {file_path} dosyasında bir hata oluştu: {e}")
        return None

def predict_gender_from_wav(file_path, model, scaler):
    """Bir WAV dosyasını kullanarak cinsiyet tahmini yapar."""
    features = extract_features(file_path)
    if features is not None:
        features_scaled = scaler.transform([features])
        prediction = model.predict(features_scaled)
        # XGBoost tahminini 'male' ve 'female' olarak döndürmek için
        return "erkek" if prediction[0] == 1 else "kadın"
    else:
        return "Tahmin yapılamadı."


# --- Veri Yükleme ve Ön İşleme ---
print("Veri setini yüklüyor ve öznitelikleri çıkarıyor...")

all_features = []
labels = []
label_map = {'male': 1, 'female': 0}

# Sizin belirttiğiniz klasör yapısına göre döngü
for gender in ['male', 'female']:
    gender_path = os.path.join(data_path, gender)
    if os.path.isdir(gender_path):
        for file_name in os.listdir(gender_path):
            if file_name.endswith('.wav'):
                file_path = os.path.join(gender_path, file_name)
                features = extract_features(file_path)
                if features is not None:
                    all_features.append(features)
                    labels.append(label_map[gender])
    else:
        print(f"Uyarı: {gender_path} klasörü bulunamadı. Lütfen veri setini doğru yüklediğinizden emin olun.")

if len(all_features) == 0:
    print("Hiçbir ses dosyası bulunamadı. Lütfen veri klasörü yapısını kontrol edin.")
else:
    X = np.array(all_features)
    y = np.array(labels)

    # Verileri ölçeklendirme
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Eğitim ve test setlerini ayırma
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

    # --- Model Eğitimi ---
    print("XGBoost modelini eğitiyor...")
    model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
    model.fit(X_train, y_train)

    # Modeli ve scaler'ı kaydetme (gelecekte kullanmak için)
    joblib.dump(model, 'xgboost_gender_model.pkl')
    joblib.dump(scaler, 'scaler.pkl')

    print("Model eğitimi tamamlandı.")

    # --- Yeni bir ses dosyası ile tahmin yapma ---
    print("\n--- Kendi WAV dosyanızı yükleyerek tahmin yapın ---")
    print("Bir .wav dosyası seçmek için dosyayı Colab'a yükleyin.")
    uploaded = files.upload()

    for filename in uploaded.keys():
        print(f"'{filename}' dosyası yüklendi.")
        prediction = predict_gender_from_wav(filename, model, scaler)
        print(f"Yüklenen ses dosyasının cinsiyet tahmini: {prediction}")

Veri setini yüklüyor ve öznitelikleri çıkarıyor...


/usr/local/lib/python3.12/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=512 is too large for input signal of length=395
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=512 is too large for input signal of length=354
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=512 is too large for input signal of length=463
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=512 is too large for input signal of length=408
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=512 is too large for input signal of length=489
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=512 is too large for input signal of length=439
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/librosa/core/spectrum.py:266: UserWa

XGBoost modelini eğitiyor...


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [13:56:31] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Model eğitimi tamamlandı.

--- Kendi WAV dosyanızı yükleyerek tahmin yapın ---
Bir .wav dosyası seçmek için dosyayı Colab'a yükleyin.


Saving arctic_a0001(6).wav to arctic_a0001(6).wav
'arctic_a0001(6).wav' dosyası yüklendi.
Yüklenen ses dosyasının cinsiyet tahmini: erkek


In [ ]:
from sklearn.metrics import accuracy_score

# --- Model Değerlendirmesi ---
print("\n--- Model Performansı ---")

# Test seti üzerinde tahmin yapın
y_pred = model.predict(X_test)

# Doğruluk (Accuracy) değerini hesaplayın
accuracy = accuracy_score(y_test, y_pred)

print(f"Modelin test seti üzerindeki doğruluk değeri: {accuracy:.4f}")


--- Model Performansı ---
Modelin test seti üzerindeki doğruluk değeri: 0.9985


In [ ]:

print("Bir .wav dosyası seçmek için dosyayı Colab'a yükleyin.")
uploaded = files.upload()

for filename in uploaded.keys():
   print(f"'{filename}' dosyası yüklendi.")
   prediction = predict_gender_from_wav(filename, model, scaler)
   print(f"Yüklenen ses dosyasının cinsiyet tahmini: {prediction}")

Bir .wav dosyası seçmek için dosyayı Colab'a yükleyin.


Saving i-am-so-excited-181436.wav to i-am-so-excited-181436.wav
'i-am-so-excited-181436.wav' dosyası yüklendi.
Yüklenen ses dosyasının cinsiyet tahmini: erkek


/usr/local/lib/python3.12/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=1024 is too large for input signal of length=861
  warnings.warn(
